# SUMÁRIO DO PROJETO




1. Qual o contexto da problemática? O que queremos resolver?
  - Empresa de Streamming de Música: Ela tem uma abordagem reativa em relação a clientes que não renovam a assinatura oferecendo 3 meses grátis para os que não renovam. Foi entendido que uma abordagem proativa pode ser mais efetiva.

2. Quais DADOS temos para agregar na solução?
  - Temos um dicionario de dados a seguir.

3. Quais INFORMAÇÕES podemos extrair desses DADOS?
  - aqui irei analisar as possiveis variaveis que podem ser criadas (Feature Engineering)

4. Como chegar na solução e quais métodos/tecnologias podem apoiar e quais não podem? (justificação dos usos, questões de negócio, performance, processamento computacional, recursos e implantação em uma hipotética produtização)
  - aqui escrevo sobre ....

5. Ótica de negócios: Ganhos $$
  - a


para encaixar no sumário:

1- EDA
- hipoteses voltadas ao problema e para a modelagem
- implicações das análises
- usar o EDA para tomada de decisões

2- Dataprep + feature engeneering + feature selection
- criação do publico-alvo, target, análise de volumetrias, missing, outliers, etc.
- construção de novas features / transformação de features
- feature selection adequado, variaveis significativas para o problema

3- Modelagem
- Estrutura do modelo utilizado, funcionamento, hiperparametros, vantagens e fraquezas
- Modelagem e pipe line adequados ao problema? eu entendo isso?
- Compreendo os impactos ao utilizar outras tecnicas de modelagem e pipeline construidos?

4- Validação
- Validação adequada? (Leakage, overfit, underfit, tamanho da amostra)
- Formas de validação exploradas? (cross-validation, train test split, out of sample, out of time, etc)
- Como as alterações do problema impactam na validação escolhida?

5- Definição e avaliação dos resultados
- Métricas de avaliação adequadas? (MAE, RMSE, Recall, AUC, Silhueta, etc)
- Como outras métricas impactariam os resultados do case?
- Como mudanças na estrutura do problema impactariam diferentes métricas?

6- Qualidade do código
- Organização e boas praticas do codigo (nomes intuitivos, comentários, ordenamento intuitivo)
- Codigo padronizado? (classes, funções e variaveis com nomes padronizados, clean code, pep8, etc?)
- Código visa a implementação? (modularização, tempo de processamento, versionamento etc?)

7- Soft Skills
- Sucinto e dentro do tempo indicado
- Clareza nas explicações
- Apresentação estruturada (começo, meio e fim)


# Instalação do ambiente inicial

Conectando com o Google Drive (armazenamento)

Instalação do Java

Instalação do Apache Spark

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark

# Configurando o Spark

Configuração das variaveis ambiente

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

# tornar o pyspark "importável"
import findspark
findspark.init()

# iniciar uma sessão local
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('Case_dataMaster_Victor').getOrCreate()

#Importando os dados da RAW

Dados baixados e localizados no google drive para nao tem repetição em acessar o ambiente do kaggle e baixar os dados de la

In [ ]:
df_members = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/data_master_VICTOR/raw/members.parquet')
df_transactions = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/data_master_VICTOR/raw/transactions.parquet')
df_userlogs = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/data_master_VICTOR/raw/user_logs.parquet')

# Inicio de EDA (Exploratory Data and Analysis)



## Dicionario de dados INICIAL

### 1. Transactions:
  - Colunas:
    - 1º 'mnso': id do usuário;
    - 2º 'payment_method_id': Método de pagamento;
    - 3º 'payment_plan_days': tamanho do plano em DIAS;
    - 4º 'plan_list_price': preço do respectivo plano em New Taiwan Dollar (NTD);
    - 5º 'actual_amount_paid': Total pago atual (nao entendi);
    - 6º 'is_auto_renew': Renovação automatica 1 ou 0;
    - 7º 'transaction_date': data da transação no formato %Y %M %D;
    - 8º 'membership_expire_date': quando o plano vence no formato %Y %M %D;
    - 9º 'is_cancel': se nessa transação o cliente cancelou ou não o plano;
    - 10º 'safra': data (Data foto da base).

### 2. User_logs:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'safra': formato %Y %M %D (data foto da base);
    - 3º 'num_25': quantidade de músicas tocadas menos de 25% da duração dela;
    - 4º 'num_50': quantidade de músicas tocadas entre 25% e 50% da duração dela;
    - 5º 'num_75': quantidade de músicas tocadas entre 50% e 75% da duração dela;
    - 6º 'num_985': quantidade de músicas tocadas entre 75% e 98,5% da duração dela;
    - 7º 'num_100': quantidade de músicas tocadas acima de 98,5%;
    - 8º 'num_unq': quantidade de musicas tocadas apenas 1 vez;
    - 9º 'total_secs': total de segundos tocados.

### 3. Members:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'city': cidade do usuário?
    - 3º 'bd': idade? obs: outliers em 2015
    - 4º 'gender': gênero
    - 5º 'registered_via': meio de registro do cliente
    - 6º 'registration_init_time': primeiro regristo do cliente formato %Y %M %D
    - 7º 'safra': Data da foto da base
    - 8º 'is_ativo': se o cliente esta ativo ou não

## Importando bibliotecas e funções necessárias para a exploração

In [ ]:
from pyspark.sql.functions import col, countDistinct, min, max, when
import matplotlib.pyplot as plt

## Entendendo o formato dos dados

In [ ]:
print("Formato da transactions:", df_transactions.count(), "X", len(df_transactions.columns))
df_transactions.printSchema()

print("Formato da user_logs:", df_userlogs.count(), "X", len(df_userlogs.columns))
df_userlogs.printSchema()

print("Formato da members: ", df_members.count(), "X", len(df_members.columns))
df_members.printSchema()

Formato da transactions: 20712225 X 10
root
 |-- msno: string (nullable = true)
 |-- payment_method_id: string (nullable = true)
 |-- payment_plan_days: string (nullable = true)
 |-- plan_list_price: string (nullable = true)
 |-- actual_amount_paid: string (nullable = true)
 |-- is_auto_renew: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- membership_expire_date: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- safra: integer (nullable = true)

Formato da user_logs: 26758971 X 9
root
 |-- msno: string (nullable = true)
 |-- safra: integer (nullable = true)
 |-- num_25: double (nullable = true)
 |-- num_50: double (nullable = true)
 |-- num_75: double (nullable = true)
 |-- num_985: double (nullable = true)
 |-- num_100: double (nullable = true)
 |-- num_unq: double (nullable = true)
 |-- total_secs: double (nullable = true)

Formato da members:  63867246 X 8
root
 |-- msno: string (nullable = true)
 |-- safra: string (nullable = true

## 1 - Entendendo a Members

### Primeira visão

In [ ]:
df_members.show(5)

+--------------------+------+----------------------+----+---+------+--------------+--------+
|                msno| safra|registration_init_time|city| bd|gender|registered_via|is_ativo|
+--------------------+------+----------------------+----+---+------+--------------+--------+
|+++snpr7pmobhLKUg...|201612|              20140927|   1|  0|  null|             7|       1|
|++/AwGzubug3gT6J+...|201607|              20150322|   1|  0|  null|             9|       0|
|++/Gw1B9K+XOlB3hL...|201601|              20121217|  15| 32|  male|             3|       1|
|++02XbtviomSxcIBU...|201603|              20131112|  14| 21|  male|             7|       0|
|++0O0Bq04sB/9ZcOS...|201610|              20141021|   5| 33|  male|             3|       0|
+--------------------+------+----------------------+----+---+------+--------------+--------+
only showing top 5 rows



nas primeiras 5 linhas ja vemos a idade 0, valores nulos

### Min e Max de datas das colunas de datas para ver até onde vão

In [ ]:
df_members.select(min('registration_init_time'), max('registration_init_time')).show()
df_members.select(min('bd'), max('bd')).show()

+---------------------------+---------------------------+
|min(registration_init_time)|max(registration_init_time)|
+---------------------------+---------------------------+
|                   20040326|                   20161231|
+---------------------------+---------------------------+

+-------+-------+
|min(bd)|max(bd)|
+-------+-------+
|    -10|    994|
+-------+-------+



Temos clientes que foram registrados 20161231

In [ ]:
df_members.select(min('safra'), max('safra')).show()


+----------+----------+
|min(safra)|max(safra)|
+----------+----------+
|    201601|    201612|
+----------+----------+

+----------+----------+
|min(safra)|max(safra)|
+----------+----------+
|    201501|    201702|
+----------+----------+

+----------+----------+
|min(safra)|max(safra)|
+----------+----------+
|    201501|    201702|
+----------+----------+



Temos uma base de usuarios de referente ao ano (safra) de 2016. Conseguimos ver o comportamento desses clientes no ano de 2015, como se comportaram em 2016, até fevereiro de 2017, total de 26 meses de comportamento.

### Entendendo a quantidade de usuarios ativos em cada safra

In [ ]:
df_members.filter(col('is_ativo') == 1) \
          .groupBy('safra') \
          .agg(countDistinct('msno').alias('usuarios_ativos')) \
          .orderBy('safra') \
          .show()

+------+---------------+
| safra|usuarios_ativos|
+------+---------------+
|201601|         888663|
|201602|         918116|
|201603|         871651|
|201604|         831565|
|201605|         849653|
|201606|         844884|
|201607|         967936|
|201608|         980643|
|201609|         991285|
|201610|        1048557|
|201611|        1071113|
|201612|         978799|
+------+---------------+



### Entendendo os valores de cada coluna da Members

In [ ]:
for i in df_members.columns:
    print(i)
    df_members.select(i) \
              .agg(countDistinct(i).alias('valores_distintos')) \
              .show()

msno
+-----------------+
|valores_distintos|
+-----------------+
|          6287789|
+-----------------+

safra
+-----------------+
|valores_distintos|
+-----------------+
|               12|
+-----------------+

registration_init_time
+-----------------+
|valores_distintos|
+-----------------+
|             4663|
+-----------------+

city
+-----------------+
|valores_distintos|
+-----------------+
|               21|
+-----------------+

bd
+-----------------+
|valores_distintos|
+-----------------+
|              385|
+-----------------+

gender
+-----------------+
|valores_distintos|
+-----------------+
|                2|
+-----------------+

registered_via
+-----------------+
|valores_distintos|
+-----------------+
|               17|
+-----------------+

is_ativo
+-----------------+
|valores_distintos|
+-----------------+
|                2|
+-----------------+



###  Observações e Questionamentos

 - Clientes cadastrados a mais de 10 anos (4663 dias distintos)
 - 21 cidades, avaliar se tem correlação com Churn
 - 385 idades distintas? algo esta muito errado.
 - 2 generos e possue valores nulos
 - 17 tipos de jeitos de se registrar diferentes



### Avaliando as idades 'bd'

In [ ]:
df_members_idade = df_members.select('bd').withColumn('bd', col('bd').cast('int'))
print(df_members_idade.count())
df_members_idade.printSchema()

63867246
root
 |-- bd: integer (nullable = true)



In [ ]:
df_members_idade.select(min('bd'), max('bd')).show()

+-------+-------+
|min(bd)|max(bd)|
+-------+-------+
|  -7168|   2016|
+-------+-------+



#### Entendendo brevemente a distribuição dos dados

In [ ]:
menor_que_0 = df_members_idade.filter(col('bd') <= 0).count()
maior_que_120 = df_members_idade.filter(col('bd') >= 120).count()
entre_0_e_120 = df_members_idade.filter((col('bd') > 0) & (col('bd') < 120)).count()

print(f"Menor que 0: {menor_que_0}")
print(f"Maior que 100: {maior_que_120}")
print(f"Entre 0 e 120: {entre_0_e_120}")

Menor que 0: 39512942
Maior que 100: 4220
Entre 0 e 120: 24350084


In [ ]:
faixa_menor_que_0 = df_members_idade.filter(col('bd') <= 0).count()
faixa_0_a_10 = df_members_idade.filter((col('bd') >= 0) & (col('bd') <= 10)).count()
faixa_11_a_20 = df_members_idade.filter((col('bd') >= 11) & (col('bd') <= 20)).count()
faixa_21_a_30 = df_members_idade.filter((col('bd') >= 21) & (col('bd') <= 30)).count()
faixa_31_a_40 = df_members_idade.filter((col('bd') >= 31) & (col('bd') <= 40)).count()
faixa_41_a_70 = df_members_idade.filter((col('bd') >= 41) & (col('bd') <= 70)).count()
faixa_71_a_90 = df_members_idade.filter((col('bd') >= 71) & (col('bd') <= 90)).count()
faixa_91_a_122 = df_members_idade.filter((col('bd') >= 91) & (col('bd') <= 122)).count()
faixa_maior_que_122 = df_members_idade.filter(col('bd') >= 122).count()

print(f"Menor que 0: {faixa_menor_que_0}")
print(f"Entre 0 e 10: {faixa_0_a_10}")
print(f"Entre 11 e 20: {faixa_11_a_20}")
print(f"Entre 21 e 30: {faixa_21_a_30}")
print(f"Entre 31 e 40: {faixa_31_a_40}")
print(f"Entre 41 e 70: {faixa_41_a_70}")
print(f"Entre 71 e 90: {faixa_71_a_90}")
print(f"Entre 91 e 122: {faixa_91_a_122}")
print(f"Maior que 122: {faixa_maior_que_122}")



Menor que 0: 39512942
Entre 0 e 10: 39521396
Entre 11 e 20: 4513991
Entre 21 e 30: 10405874
Entre 31 e 40: 5798450
Entre 41 e 70: 3491374
Entre 71 e 90: 52617
Entre 91 e 122: 76378
Maior que 122: 4148


### OutLiers identificados, Colunas com tipos diferentes identificados sendo necessário transformar para a bronze

Outliers na 'bd'. Possiveis soluções para os outliers: Exclusão, substituição. Analisar a correlação de churn com a idade e ver sua importancia. Para o momento: alterações esperadas para a construção da bronze, agora apenas sinalizar idades invalidas e que precisam ser tratadas.

#### Documentar:
Quantos valores foram removidos ou corrigidos?
Que critérios foram usados para identificar os valores inválidos?
Qual o impacto de substituir ou excluir? A quantidade de substituição é grande ou pequena?

## 2 - Entendendo a Transaction

### Primeira visão

In [ ]:
df_transactions.show(10)

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel| safra|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+------+
|+++IZseRRiQS9aaSk...|               38|              410|           1788|              1788|            0|        20151121|              20170104|        0|201511|
|+++snpr7pmobhLKUg...|               41|               30|            149|               149|            1|        20150526|              20150626|        0|201505|
|+++snpr7pmobhLKUg...|               41|               30|            149|               149|            1|        20150926|              20151026|        0|201509|
|++/9R3sX3

### Entendendo cada coluna da Transaction

In [ ]:
for i in df_transactions.columns:
    print(i)
    df_transactions.select(i) \
              .agg(countDistinct(i).alias('valores_distintos')) \
              .show()

msno
+-----------------+
|valores_distintos|
+-----------------+
|          2363626|
+-----------------+

payment_method_id
+-----------------+
|valores_distintos|
+-----------------+
|               40|
+-----------------+

payment_plan_days
+-----------------+
|valores_distintos|
+-----------------+
|               37|
+-----------------+

plan_list_price
+-----------------+
|valores_distintos|
+-----------------+
|               51|
+-----------------+

actual_amount_paid
+-----------------+
|valores_distintos|
+-----------------+
|               57|
+-----------------+

is_auto_renew
+-----------------+
|valores_distintos|
+-----------------+
|                2|
+-----------------+

transaction_date
+-----------------+
|valores_distintos|
+-----------------+
|              790|
+-----------------+

membership_expire_date
+-----------------+
|valores_distintos|
+-----------------+
|             1534|
+-----------------+

is_cancel
+-----------------+
|valores_distintos|
+-----------

### Entendendo os minimos e máximos das colunas de data

In [ ]:
df_transactions.select(min('safra'), max('safra')).show()
df_transactions.select(min('transaction_date'), max('transaction_date')).show()
df_transactions.select(min('membership_expire_date'), max('membership_expire_date')).show()


+----------+----------+
|min(safra)|max(safra)|
+----------+----------+
|    201501|    201702|
+----------+----------+

+---------------------+---------------------+
|min(transaction_date)|max(transaction_date)|
+---------------------+---------------------+
|             20150101|             20170228|
+---------------------+---------------------+

+---------------------------+---------------------------+
|min(membership_expire_date)|max(membership_expire_date)|
+---------------------------+---------------------------+
|                   19700101|                   20170331|
+---------------------------+---------------------------+



### Entendendo os preços dos planos

In [ ]:
df_transactions_plan_price = df_transactions.select('plan_list_price').withColumn('plan_list_price', col('plan_list_price').cast('int'))
df_transactions_amount_paid = df_transactions.select('actual_amount_paid').withColumn('actual_amount_paid', col('actual_amount_paid').cast('int'))
df_transactions_comparacao = df_transactions.withColumn('comparacao', when(col('plan_list_price') == col('actual_amount_paid'), 1).otherwise(0))


df_transactions_plan_price.select(min('plan_list_price'), max('plan_list_price')).show()
df_transactions_amount_paid.select(min('actual_amount_paid'), max('actual_amount_paid')).show()
df_transactions_comparacao.filter(col('comparacao') == 1).count()

+--------------------+--------------------+
|min(plan_list_price)|max(plan_list_price)|
+--------------------+--------------------+
|                   0|                2000|
+--------------------+--------------------+

+-----------------------+-----------------------+
|min(actual_amount_paid)|max(actual_amount_paid)|
+-----------------------+-----------------------+
|                      0|                   2000|
+-----------------------+-----------------------+



19114847

In [ ]:
df_transactions_plan_price.describe().show()
df_transactions_amount_paid.describe().show()

+-------+------------------+
|summary|   plan_list_price|
+-------+------------------+
|  count|          20712225|
|   mean|140.24088353617248|
| stddev|132.27679209056055|
|    min|                 0|
|    max|              2000|
+-------+------------------+

+-------+------------------+
|summary|actual_amount_paid|
+-------+------------------+
|  count|          20712225|
|   mean|142.83468555406287|
| stddev|133.60944276809502|
|    min|                 0|
|    max|              2000|
+-------+------------------+



### Observações e questionamentos

  - Data de vencimento do plano em 1970?
  - Transações de 201501 a 201702
  - 40 meios de pagamentos
  - Maior parte dos registros de preços dos planos, tanto Amount quanto por plano estão com valores iguais.
  - Tipos dos dados estão atrapalhando as consultas em cima dos dados
  - Coluna is_cancel: entender como ela pode ajudar no nosso contexto

## 3 - Entendendo a User_logs

### Primeira visão

In [ ]:
df_userlogs.show(5)

+--------------------+------+------+------+------+-------+-------+-------+----------+
|                msno| safra|num_25|num_50|num_75|num_985|num_100|num_unq|total_secs|
+--------------------+------+------+------+------+-------+-------+-------+----------+
|SwlrSivYHoKF9V5wm...|201701| 121.0|  28.0|  14.0|   29.0|  704.0|  827.0|184606.903|
|rE5wSmHEF1Dhu55zh...|201605|  26.0|   2.0|   5.0|    6.0|  462.0|  256.0|119439.485|
|hx+cyaQ/Jcdr/Z5fo...|201611| 161.0|  71.0|  49.0|   34.0|  668.0|  891.0|204791.242|
|53QW6B70J23X2UCvx...|201502|  37.0|   9.0|   3.0|    9.0|  408.0|  447.0|101186.041|
|/0S1N/oRyxGLZlzxn...|201506| 205.0|  49.0|  23.0|   21.0|  225.0|  489.0| 69957.524|
+--------------------+------+------+------+------+-------+-------+-------+----------+
only showing top 5 rows



### Entendendo cada coluna da User_logs

Aqui temos somente variaveis númnericas, nenhuma categorica, nao tendo importância ver a distinção dos valores.

In [ ]:
df_userlogs.describe().show()

+-------+--------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+--------------------+
|summary|                msno|            safra|           num_25|           num_50|            num_75|           num_985|          num_100|           num_unq|          total_secs|
+-------+--------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+--------------------+
|  count|            26758971|         26758971|         26758971|         26758971|          26758971|          26758971|         26758971|          26758971|            26758971|
|   mean|                null|201572.0403868669|95.42601163549973|24.03566975725636|14.903521327483034|16.538671797207748|450.1598216538297|440.91930526775485|-2.11717488634742...|
| stddev|                null|61.87618074610147|175.2882595645691|39.12554591867812|  22.626721

In [ ]:
df_userlogs.select(min('safra'), max('safra')).show()

+----------+----------+
|min(safra)|max(safra)|
+----------+----------+
|    201501|    201702|
+----------+----------+



### Entendento o total de segundos ouvidos

In [ ]:
df_userlogs_total_secs = df_userlogs.select('total_secs')
df_userlogs_total_secs.show(5)

df_userlogs_total_secs.select(min('total_secs'), max('total_secs')).show()

df_userlogs_total_secs.describe().show()

+----------+
|total_secs|
+----------+
|184606.903|
|119439.485|
|204791.242|
|101186.041|
| 69957.524|
+----------+
only showing top 5 rows

+--------------------+--------------------+
|     min(total_secs)|     max(total_secs)|
+--------------------+--------------------+
|-2.39807672957245...|9.223372037135594E15|
+--------------------+--------------------+

+-------+--------------------+
|summary|          total_secs|
+-------+--------------------+
|  count|            26758971|
|   mean|-2.11717488634742...|
| stddev|6.839043702349416E14|
|    min|-2.39807672957245...|
|    max|9.223372037135594E15|
+-------+--------------------+



#### Analisando em horas e dias ouvidos

In [ ]:
df_userlogs_total_secs_horas = df_userlogs_total_secs.withColumn('horas', col('total_secs') / 3600)
df_userlogs_total_secs_horas.select(min('horas'), max('horas')).show()
df_userlogs_total_secs_dias = df_userlogs_total_secs.withColumn('dias', col('total_secs') / 86400)
df_userlogs_total_secs_dias.select(min('dias'), max('dias')).show()

+--------------------+--------------------+
|          min(horas)|          max(horas)|
+--------------------+--------------------+
|-6.66132424881237...|2.562047788093220...|
+--------------------+--------------------+

+--------------------+--------------------+
|           min(dias)|           max(dias)|
+--------------------+--------------------+
|-2.77555177033848...|1.067519911705508...|
+--------------------+--------------------+



### Observações e Questionamentos

  - Tipo do dado da data integer, indo de 201501 a 201702
  - Dados como Double
  - Valores negativos para segundos totais de musicas ouvidas? Valores discrepantes para total de segs ouvidos
  - Não há valores faltantes

## 4 - Entendendo o Conjunto. Como são as DataBases e como elas se relacionam?



  1. Members: Registro de todos os usuários do sistema, contém daonde o usuário é, idade, genero, como ele se registrou e qual foi a data que ele entrou no sistema e se ele esta ativo ou não (assinante ou não).

    - **Desafio**: Lidar com a idade dos clientes

  2. Transactions: Transações de todos os clientes quando assinam o plano, contendo data de assinatura, tempo para vencimento, meios de pagamento, valores do plano, total até agora arrecadado de cada cliente. Aqui os clientes se repetiram com uma visão acumulada de casa transação por data.

    - **Desafio**: Lidar com o preço dos planos assinados

  3. User_logs: Logs de todos os usuários, contendo seus comportentos dentro do sistema, com tempo total de músicas ouvidas, e parcelas de músicas ouvidas, podendo ser identificado possiveis musicas preferidas ou não.

    - **Desafio**: Lidar com o total de segundos ouvidos

## Dicionario de dados FINAL

1. Transactions:
  - Colunas:
    - 1º 'mnso': id do usuário (string);
    - 2º 'payment_method_id': Método de pagamento, Númerico (int);
    - 3º 'payment_plan_days': tamanho do plano em DIAS (int);
    - 4º 'plan_list_price': preço do respectivo plano em New Taiwan Dollar (NTD) (Float/Double);
    - 5º 'actual_amount_paid': Total pago atual (nao entendi) (Float/Double);
    - 6º 'is_auto_renew': Renovação automatica (1 ou 0) (int);
    - 7º 'transaction_date': data da transação no formato %Y %M %D (?);
    - 8º 'membership_expire_date': quando o plano vence no formato %Y %M %D;
    - 9º 'is_cancel': se nessa transação o cliente cancelou ou não o plano;
    - 10º 'safra': data (nao entendi 100% ao que se refere essa data).

2. User_logs:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'safra': formato %Y %M %D (nao entendi essa data);
    - 3º 'num_25': quantidade de músicas tocadas menos de 25% da duração dela;
    - 4º 'num_50': quantidade de músicas tocadas entre 25% e 50% da duração dela;
    - 5º 'num_75': quantidade de músicas tocadas entre 50% e 75% da duração dela;
    - 6º 'num_985': quantidade de músicas tocadas entre 75% e 98,5% da duração dela;
    - 7º 'num_100': quantidade de músicas tocadas acima de 98,5%;
    - 8º 'num_unq': quantidade de musicas tocadas apenas 1 vez;
    - 9º 'total_secs': total de segundos tocados.

3. Members:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'city': cidade do usuário, Númerico;
    - 3º 'bd': idade, obs: outliers em 2015;
    - 4º 'gender': gênero, Categorico (male or female);
    - 5º 'registered_via': meio de registro do cliente, Númerico;
    - 6º 'registration_init_time': primeiro regristo do cliente formato %Y %M %D, de 2004 a 20161231;
    - 7º 'safra': base do inicio ao fim de 2016;
    - 8º 'is_ativo': se o cliente esta ativo ou não, 0 ou 1;

## Analise de distribuição das variaveis

 - Analise inicial para entender como os valores estão em termo de frequencia, afim de detectar alguma anomalidade, como outliers

## Analise de valores missing

## Checagem de integridade da Base

 - Valores duplicados:
 - Missing:
 - Consistencia:

# Construção da Bronze:

- Limpeza e tratamento dos dados

    - Input dos ausentes, se necessário
    - Remoção de outliers, se necessário
    - Normalização e padronização, se necessário
    - Conversão de tipos de dados

- Feature Engineering
    - novas variaveis

# Construção da Silver:

- Analise de Features e seleção
- Construção da Target

# Construção da Gold

- Analytical Base Table (ABT)
- Features e Target estruturados e prontos para irem para treino, validação e teste dos modelos

# Criação dos modelos